In [ ]:
import pandas as pd
import os
import glob
import numpy as np

### CAMBIO PATH PER PRENDERE I FILE CSV PER LA CREAZIONE DEL DATAFRAME

In [ ]:
path_file = os.path.abspath(os.getcwd()) #prendiamo il path in cui si trova il file su cui stiamo lavorando
print(path_file)
os.chdir(path_file) #cambiamo directory al fine di poter prendere i file csv per la creazione del dataframe
data = "\dataset"

In [ ]:
final_path = path_file + data
print(final_path)

In [ ]:
data_path = os.path.join(final_path,'*.csv') #lista di tutti gli elementi con estensione csv nella cartella final_path
csv_list = glob.glob(data_path) #converte data path in un output Unix-like (ls) (*.csv -> lista di elementi con estensione csv)
print(csv_list)

###  DATAFRAME CHE CONTERRA' I DATI

In [ ]:
df = pd.DataFrame() #creazione di un dataframe vuoto

count = 0
# ciclo per scorrere tutti i csv
for csv_file in csv_list: # ciclo che scorre i csv nella cartella path
    df_csv = pd.read_csv(csv_file) 
    if count == 3:
        first = ['Dst Port', 'Protocol', 'Timestamp','Flow ID','Src IP','Dst IP']
    else:
        first = ['Dst Port', 'Protocol', 'Timestamp']
    
    for a in first:
        df_csv.drop(columns=a,axis=1,inplace=True)
    df = pd.concat([df,df_csv],ignore_index=True)
    count = count + 1

pd.set_option('display.max_columns', None)
display(df)

In [ ]:
# Eliminiamo tutte le feature che hanno un solo valore

scarto = []
scarto.append([])

for a in df.head(0):
    
    if(df[a].unique().shape[0] == 1):
        scarto[-1].append(a)
        df.drop(columns=a,axis=1,inplace=True)
    else:
        continue

In [ ]:
#rimuovo tutte le righe che sono una duplicazione dell'intestazione
df = df[df.Label != "Label"]
print(df.shape)

In [ ]:
attaccanti_label = df['Label'].unique()
print(attaccanti_label)

In [ ]:
# sostituisco tutte le label diverse da "benign" con "malign" per avere un clustering più efficiente

for i in range(len(df)):
    if df.loc[i, 'Label'] != 'Benign':
        df.loc[i, 'Label'] = 'Malign'

df

In [ ]:
attaccanti_label = df['Label'].unique()
print(attaccanti_label)

In [ ]:
# Andiamo a sostituire le label con valori interi

for b,c in zip(attaccanti_label,range(len(attaccanti_label))):
    df = df.replace(b, c)

In [ ]:
# Nel documento sono presenti valori infiniti, li andiamo a sostituire con Nan che verranno successivamente rimossi

df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Eliminiamo i valori nulli
df.dropna(inplace=True)

In [ ]:
# Andiamo a castare tutte le stringhe rimanenti in numeri, in quanto alcuni numeri sono rappresentati come stringhe

for label in df.head(0):
    if(label != 'Label'):
        try:
            df[label] = df[label].astype(float)
        except:
            print(label)

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
# Definisco una funzione per normalizzare i valori

def normalize(feature:str):
    return (df[feature]-df[feature].min())/(df[feature].max()-df[feature].min())


# Andiamo a normalizzare i valori per poter usare i vari algoritmi di feature selection

scaled_document = df.head(0)

for a in df.head(0):
    
    try:
        if df[a].max() != 0 and a != 'Label':
            scaled_document[a] = normalize(a)
        else:
            scaled_document[a] = df[a]
    except:

        print(a)

mi devo salvare il pickle (fare in questo file)
poi riprendere i file dei feature selection per vedere quali sono le 5 feature più importanti (si può fare in un altro file)